In [1]:
class Range:
    destination = 0
    source = 0
    length = 0
    start = 0
    end = 0
    def __init__(self, destination, source, length):
        self.destination = destination
        self.source = source
        self.length = length

    def contains(self, value):
        return self.source <= value < self.source + self.length

    def transform(self, value):
        if not self.contains(value):
            return None
        return self.destination + value - self.source

    def __repr__(self):
        if self.destination == self.source:
            delta = ""
        elif self.destination > self.source:
            delta = f"+{self.destination-self.source}"
        else:
            delta = f"{self.destination-self.source}"
        return f"[{self.source}:{self.source + self.length - 1}]{delta}"

In [2]:
class Interval:
    start = 0
    end = 0

    def __init__(self, start, end):
        self.start = start
        self.end = end

    @staticmethod
    def from_range(range):
        return Interval(range.source, range.source + range.length - 1)
    
    def __repr__(self):
        return f"[{self.start}:{self.end}]"

In [3]:
class Map:
    from_type = None
    to_type = None
    ranges = []
    def __init__(self, from_type=None, to_type=None, ranges=None):
        self.from_type = from_type
        self.to_type = to_type
        self.ranges = ranges or []

    def transform(self, value):
        for r in self.ranges:
            v = r.transform(value)
            if v:
                return v
        return value

    def __repr__(self):
        return repr(self.__dict__)

In [4]:
def get_input(fname="test.txt"):
    seeds = []
    maps = [Map()]
    with open(fname) as f:
        lines = f.readlines()
        seeds = [int(i) for i in lines[0].rstrip().split(": ")[-1].split(" ")]
        for l in lines[2:]:
            line = l.rstrip()
            if not line:
                maps.append(Map())
                continue
            if line[-1] == ":":
                parts = line.split(" ")[0].split("-")
                maps[-1].from_type = parts[0]
                maps[-1].to_type = parts[-1]
                continue
            values = [int(i) for i in line.split(" ")]
            maps[-1].ranges.append(Range(values[0], values[1], values[2]))
    return seeds, maps

In [5]:
test_seeds, test_maps = get_input("test.txt")
my_seeds, my_maps = get_input("input.txt")

In [6]:
test_seeds

[79, 14, 55, 13]

In [7]:
test_maps

[{'from_type': 'seed', 'to_type': 'soil', 'ranges': [[98:99]-48, [50:97]+2]},
 {'from_type': 'soil', 'to_type': 'fertilizer', 'ranges': [[15:51]-15, [52:53]-15, [0:14]+39]},
 {'from_type': 'fertilizer', 'to_type': 'water', 'ranges': [[53:60]-4, [11:52]-11, [0:6]+42, [7:10]+50]},
 {'from_type': 'water', 'to_type': 'light', 'ranges': [[18:24]+70, [25:94]-7]},
 {'from_type': 'light', 'to_type': 'temperature', 'ranges': [[77:99]-32, [45:63]+36, [64:76]+4]},
 {'from_type': 'temperature', 'to_type': 'humidity', 'ranges': [[69:69]-69, [0:68]+1]},
 {'from_type': 'humidity', 'to_type': 'location', 'ranges': [[56:92]+4, [93:96]-37]}]

In [8]:
def solve(seeds, maps):
    seed_values= { 'seed': seeds }
    maps = { m.from_type: m for m in maps }
    current_type = 'seed'
    while current_type != 'location':
        previous_type = current_type
        current_type = maps[current_type].to_type
        seed_values[current_type] = [maps[previous_type].transform(s) for s in seed_values[previous_type]]
    return seed_values

In [9]:
solve(test_seeds, test_maps)

{'seed': [79, 14, 55, 13],
 'soil': [81, 14, 57, 13],
 'fertilizer': [81, 53, 57, 52],
 'water': [81, 49, 53, 41],
 'light': [74, 42, 46, 34],
 'temperature': [78, 42, 82, 34],
 'humidity': [78, 43, 82, 35],
 'location': [82, 43, 86, 35]}

In [10]:
min(solve(test_seeds, test_maps)['location'])

35

In [11]:
min(solve(my_seeds, my_maps)['location'])

282277027

In [12]:
def sorted_intervals(intervals):
    return sorted(intervals, key=lambda i: i.start)

In [13]:
def merge_intervals(intervals):
    intervals = sorted_intervals(intervals)
    merged = [Interval(intervals[0].start, intervals[0].end)]
    for i in range(1, len(intervals)):
        if intervals[i].start == merged[-1].end + 1:
            merged[-1].end = intervals[i].end
        else:
            merged.append(Interval(intervals[i].start, intervals[i].end))
    return merged

In [14]:
merge_intervals([Interval(1, 2), Interval(3, 4), Interval(10, 14)])

[[1:4], [10:14]]

In [15]:
merge_intervals([Interval(1, 2), Interval(4, 7)])

[[1:2], [4:7]]

In [16]:
Range.transform_interval = lambda self, interval: Interval(self.transform(interval.start), self.transform(interval.end))

In [17]:
def solve2(seeds, maps):
    seed_intervals = []
    for i in range(len(seeds) // 2):
        seed_intervals.append(Interval(seeds[i * 2], seeds[i * 2] + seeds[i * 2 + 1] - 1))
    seed_intervals = sorted_intervals(seed_intervals)
    maps = { m.from_type: m for m in maps }
    current_type = 'seed'
    while current_type != 'location':
        previous_type = current_type
        current_type = maps[current_type].to_type
        map_ranges = sorted(maps[previous_type].ranges, key=lambda r: r.source)
        map_intervals = [Interval.from_range(r) for r in map_ranges]
        previous = seed_intervals
        seed_intervals = []
        i = 0
        j = 0
        while i < len(previous) and j < len(map_intervals):
            while i < len(previous) and previous[i].end < map_intervals[j].start:
                seed_intervals.append(previous[i])
                i += 1
            if i >= len(previous):
                break
            # Intersection?
            # 1 - map range contains all seed range
            if map_intervals[j].start <= previous[i].start and previous[i].end <= map_intervals[j].end:
                seed_intervals.append(map_ranges[j].transform_interval(previous[i]))
                i += 1
                continue
            # 2 - seed range contains all map range
            if previous[i].start <= map_intervals[j].start and map_intervals[j].end <= previous[i].end:
                seed_intervals.append(Interval(previous[i].start, map_intervals[j].start - 1))
                seed_intervals.append(map_ranges[j].transform_interval(map_intervals[j]))
                previous[i] = Interval(map_intervals[j].end + 1, previous[i].end)
                j += 1
                continue
            # 3 - seed range overlaps to the left
            if map_intervals[j].start <= previous[i].end <= map_intervals[j].end:
                seed_intervals.append(Interval(previous[i].start, map_intervals[j].start - 1))
                seed_intervals.append(map_ranges[j].transform_interval(map_intervals[j]))                
                i += 1
                continue
            # 4 - seed range overlaps to the right
            if map_intervals[j].start <= previous[i].start <= map_intervals[j].end:
                seed_intervals.append(map_ranges[j].transform_interval(Interval(previous[i].start, map_intervals[j].end)))
                previous[i] = Interval(map_intervals[j].end + 1, previous[i].end)
                j += 1
                continue
            # 5 - skip
            j += 1
        while i < len(previous):
            seed_intervals.append(previous[i])
            i += 1
        seed_intervals = merge_intervals(seed_intervals)
    return seed_intervals

In [18]:
solve2(test_seeds, test_maps)

[[46:96], [82:84], [86:89], [93:92], [94:98]]

In [19]:
solve2(my_seeds, my_maps)[0].start

11554135